In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
import time
from tqdm import tqdm
from scipy.stats import pearsonr

print('done')

%load_ext autoreload
%autoreload 2

In [ ]:
import torch
# Check GPU availability
use_gpu = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_gpu \
        else torch.FloatTensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# small function to plot images
def showTensor(aTensor, pos=111):
    plt.subplot(int(pos))
    plt.imshow(aTensor)    #,cmap='viridis')
    plt.colorbar()
    #plt.show()
    plt.rcParams["axes.grid"] = False

In [ ]:
# Functions to create complex tensors (ie, split real & img parts) + transfer between numpy and pytorch

def to_complex_tensor(stuff):
    # TO_COMPLEX_TENSOR transforms a complex nparray into a complex tensor
    # The real and imaginary part are concatenated along the last dimension
    
    # Retrieve real and imaginary parts in separate tensors
    stuff_r = torch.from_numpy(np.real(stuff)).to(device).float()
    stuff_i = torch.from_numpy(np.imag(stuff)).to(device).float()
    
    # Add a last dimension
    stuff_r = torch.unsqueeze(stuff_r, dim=stuff_r.ndimension())
    stuff_i = torch.unsqueeze(stuff_i, dim=stuff_i.ndimension())
    
    # Return the concatenation
    return torch.cat((stuff_r, stuff_i), dim=-1)

def from_complex_tensor(stuff):
    # FROM_COMPLEX_TENSOR transforms a complex tensor into a complex nparray
    
    # Retrieve real and imaginary parts in separate tensors
    (stuff_r, stuff_i) = torch.chunk(stuff, 2, dim=-1)
    # Convert to numpy
    stuff_r = np.squeeze(stuff_r.cpu().detach().numpy())
    stuff_i = np.squeeze(stuff_i.cpu().detach().numpy())
    # Return the complex-valued nparray
    return stuff_r + 1j * stuff_i

In [ ]:
# Functions to compute complex tensors (eg: tensor muliplication) which are not implemented in Pythorch

def complex_matmul(a, b):
    # COMPLEX_MATMUL computes a@b with a and b complex on the specified device

    # Ensure that inputs are on the correct device and are in the expected format
    a = a.to(device)
    b = b.to(device)

    # Retrieve real and imaginary parts in separate tensors
    (a_r, a_i) = torch.chunk(a, 2, dim=-1)
    (b_r, b_i) = torch.chunk(b, 2, dim=-1)
    
    # Remove the last dimension
    a_r = a_r.squeeze(-1)
    a_i = a_i.squeeze(-1)
    b_r = b_r.squeeze(-1)
    b_i = b_i.squeeze(-1)
    
    # Compute the multiplication
    real_part = a_r @ b_r - a_i @ b_i
    imag_part = a_i @ b_r + a_r @ b_i
    
    # Return the concatenation, re-adding the last dimension
    result = torch.cat((real_part.unsqueeze(-1), imag_part.unsqueeze(-1)), dim=-1)

    return result

#     return torch.cat((real_part[:, np.newaxis], imag_part[:, np.newaxis]), dim=-1)

def complex_multiply(a, b):
    # COMPLEX_MULTIPLY computes a*b with a and b complex

    # Retrieve real and imaginary parts in separate tensors
    (a_r, a_i) = torch.chunk(a, 2, dim=-1)
    (b_r, b_i) = torch.chunk(b, 2, dim=-1)
    # Remove the last dimension
    a_r = torch.squeeze(a_r)
    a_i = torch.squeeze(a_i)
    b_r = torch.squeeze(b_r)
    b_i = torch.squeeze(b_i)
    # Compute the multiplication
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_i * b_r + a_r * b_i
    # Return the concatenation
    return torch.cat((real_part.unsqueeze_(-1), imag_part.unsqueeze_(-1)), dim=-1)

def complex_norm2(b):
    # COMPLEX_NORM2 computes the L2 norm square of b
    return torch.sum(b**2)

In [ ]:
def phase_to_complex(phi):
    # PHASE_TO_COMPLEX transforms a real-valued tensor into a complex tensor
    # Compute the real and imaginary parts in separate tensors    
    stuff_r = torch.cos(phi)
    stuff_i = torch.sin(phi)
    # Add a last dimension
    stuff_r = torch.unsqueeze(stuff_r, dim=stuff_r.ndimension())
    stuff_i = torch.unsqueeze(stuff_i, dim=stuff_i.ndimension())
    # Return the concatenation
    return torch.cat((stuff_r, stuff_i), dim=-1)

# Incoherent Process (eg: Fluorescence)

# Data Loader

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import struct
from scipy.ndimage import zoom

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

def resize_image(image, new_size):
    # scale factor
    scale = [n / float(o) for n, o in zip(new_size, image.shape)]
    return zoom(image, zoom=scale, order=1)

train_num = 100
val_num = 20

train_images = read_idx('MNIST/train/train-images.idx3-ubyte')

train_indices = np.random.choice(len(train_images), train_num, replace=False)
val_indices = np.random.choice(len(train_images), val_num, replace=False)

def process_images(indices, image_count):
    processed_images = []
    for index in indices[:image_count]:
        # padding
        padded_image = np.pad(train_images[index], pad_width=2, mode='constant', constant_values=0)
        # resize
        # resized_image = resize_image(padded_image, (64, 64))
        processed_images.append(padded_image)
    return processed_images

processed_train_images = process_images(train_indices, train_num)
processed_val_images = process_images(val_indices, val_num)

# first train image
plt.imshow(processed_train_images[0], cmap='gray')
plt.title('Processed Training Image')
plt.axis('off')
plt.show()

# first val image
plt.imshow(processed_val_images[0], cmap='gray')
plt.title('Processed Validation Image')
plt.axis('off')
plt.show()


### Forward Model

In [ ]:
Npat = 100 # nb of patterns on the SLM
Nslm = 1024  # nb of SLM pixels
Nfluo = 1024   # nb of beads
Ncam = 1024  # nb of camera pixels

magnitude = np.random.randn(Nfluo, Nslm)

T1 = magnitude + 1j * magnitude
T1_t = to_complex_tensor(T1)

processed_train_images_flat = [img.reshape(-1) for img in processed_train_images] 
processed_val_images_flat = [img.reshape(-1) for img in processed_val_images]

SLM_input_train = np.stack(processed_train_images_flat, axis=1) 
SLM_input_val = np.stack(processed_val_images_flat, axis=1)

# to torch tensor
SLM_input_train_pha = torch.from_numpy(SLM_input_train).float().to(device)
SLM_input_train_t = phase_to_complex(SLM_input_train_pha)

SLM_input_val_pha = torch.from_numpy(SLM_input_val).float().to(device)
SLM_input_val_t = phase_to_complex(SLM_input_val_pha)

field1_train_t = complex_matmul(T1_t, SLM_input_train_t)
int1_train_t = torch.sum(torch.abs(field1_train_t)**2,-1)

field1_val_t = complex_matmul(T1_t, SLM_input_val_t)
int1_val_t = torch.sum(torch.abs(field1_val_t)**2,-1)
# int1_val_t = int1_val_t.T
# CAM_output_t = T2_t@int1_t

In [ ]:
torch.save(T1_t.cpu(), 'MNIST/train/T1.npy')

In [ ]:
import os

# SLM_input_train was saved in NumPy 
np.save('MNIST/train/train_image.npy', SLM_input_train)

# int1_train_t is a PyTorch tensor, so load to cpu
torch.save(int1_train_t.cpu(), 'MNIST/train/train_speckle.npy')

In [ ]:
np.save('MNIST/val/val_image.npy', SLM_input_val)

torch.save(int1_val_t.cpu(), 'MNIST/val/val_speckle.npy')

In [ ]:
array = np.load('MNIST/train/train_image.npy')

print(int1_train_t)
max_element = torch.max(int1_train_t)
min_element = torch.min(int1_train_t)

print("maximum:", max_element.item())
print("minimum:", min_element.item())
print("Shape of the array:", array.shape)

In [ ]:
first_row = int1_train_t.T.cpu()[0, :]  # data saved in row
matrix_128x128 = first_row.reshape((32, 32))

plt.imshow(matrix_128x128, cmap='viridis', interpolation='none')
plt.colorbar()
plt.title('Visualization of 128x128 Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

output_dir = 'data/train/input'
os.makedirs(output_dir, exist_ok=True)

# to .tif
for idx, img in enumerate(processed_train_images):
    # normalization
    img = (img - img.min()) / (img.max() - img.min())
    img = (img * 255).astype(np.uint8)

    image = Image.fromarray(img)
    file_path = os.path.join(output_dir, f'train_image_{idx}.png')
    image.save(file_path)

print(f"Saved {len(processed_train_images)} images to {output_dir}")

file_name = 'train_image_0.png'
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray')  # image saved in grayscale
plt.title(f'Displaying {file_name}')
plt.axis('off') 
plt.show()

In [ ]:
output_dir = 'data/val/input'
os.makedirs(output_dir, exist_ok=True)

for idx, img in enumerate(processed_val_images):
    img = (img - img.min()) / (img.max() - img.min())  
    img = (img * 255).astype(np.uint8)

    image = Image.fromarray(img)
    file_path = os.path.join(output_dir, f'val_image_{idx}.png')
    image.save(file_path)

print(f"Saved {len(processed_val_images)} images to {output_dir}")

file_name = 'val_image_0.png'  
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray')  # image saved in grayscale
plt.title(f'Displaying {file_name}')
plt.axis('off')
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt
import os

speckle_data = torch.load('MNIST/train/train_speckle.npy')

output_dir = 'data/train/output'
os.makedirs(output_dir, exist_ok=True)

for i in range(speckle_data.shape[1]): 
    # save as n*n image
    image = speckle_data[:, i].reshape(32, 32).numpy() 
    
    # normalization
    image_normalized = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    # scaling
    image_scaled = (image_normalized * 255).astype(np.uint8)
    
    output_filename = os.path.join(output_dir, f'train_speckle_{i}.png')
    plt.imsave(output_filename, image_scaled, cmap='gray')

file_name = 'train_speckle_0.png'
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray', interpolation='none')
plt.colorbar() 
plt.title('Visualization of 128x128 Matrix')
plt.show()


In [ ]:
max_element = torch.max(image)
min_element = torch.min(image)

print("maximum:", max_element.item())
print("minimum:", min_element.item())

In [ ]:
speckle_data = torch.load('MNIST/val/val_speckle.npy')

output_dir = 'data/val/output'
os.makedirs(output_dir, exist_ok=True)

for i in range(speckle_data.shape[1]): 
    # save as n*n image
    image = speckle_data[:, i].reshape(32, 32).numpy()  
    
    image_normalized = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    image_scaled = (image_normalized * 255).astype(np.uint8)
    
    output_filename = os.path.join(output_dir, f'val_speckle_{i}.png')
    plt.imsave(output_filename, image_scaled, cmap='gray')

file_name = 'val_speckle_0.png' 
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray', interpolation='none')
plt.colorbar() 
plt.title('Visualization of 128x128 Matrix')
plt.show()

# Deletion

In [ ]:
import os
import shutil

dir_path = 'data/train/noised_gaussian_150_L'

if os.path.exists(dir_path):
    # Remove all files and folders in the directory
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove file or link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
else:
    print(f'The directory {dir_path} does not exist.')


# Noise

## Mask-out

In [ ]:
import numpy as np
import os
from PIL import Image

# Noise probability
p = 0.3

# Input and output directories
input_dir = 'data/train/output'
output_dir = 'data/train/noised_03'
mask_dir = os.path.join(output_dir, 'masks')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

# Process each image
for filename in image_files:
    # Read the image and convert to grayscale
    img_path = os.path.join(input_dir, filename)
    image = np.array(Image.open(img_path).convert('L'))  # 'L' mode for grayscale
    
    # Create a random matrix and apply noise
    mask = np.random.rand(*image.shape) < p
    noised_image = image.copy()
    noised_image[mask] = 0
    
    # Save the processed image
    output_img_path = os.path.join(output_dir, f'train_noise_{filename.split("_")[-1]}')
    Image.fromarray(noised_image).save(output_img_path)
    
    # Save the mask (0 where noise was added, 1 elsewhere)
    mask_save = np.where(mask, 0, 1)
    mask_filename = f'mask_{filename.split("_")[-1]}'
    mask_filename = mask_filename.replace('.png', '.npy')
    mask_img_path = os.path.join(mask_dir, mask_filename)
    np.save(mask_img_path, mask_save)  # Save the mask as a numpy array

# Assuming we want to display the first image and its mask
file_name = 'train_noise_0.png'
mask_name = 'mask_0.npy'
file_path = os.path.join(output_dir, file_name)
mask_path = os.path.join(mask_dir, mask_name)

# Load the image and the mask
image = Image.open(file_path)
mask = np.load(mask_path)  # Load the mask numpy array

# Display the image and the mask
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title(f'Displaying {file_name}')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')  # Mask might look fully white if most values are 1
plt.title(f'Displaying {mask_name}')
plt.axis('off')

plt.show()


## Gaussian

In [ ]:
import torch
import numpy as np
import os
import cv2
from PIL import Image
import scipy.io as sio

def add_gaussian_noise_and_complex_adjust(img, field_column, model_path, sigma):
    img = torch.from_numpy(img).to(torch.float32).to('cuda')  # Convert image to PyTorch tensor and move to GPU
    print(img[1, :])
    index = model_path.rfind("/")
    if sigma > 0:
        noise = torch.normal(mean=0.0, std=sigma / 255., size=img.size(), device='cuda')
        # noise = torch.clamp(noise, -0.5, 0.5)
        print(noise[1, :])
        sio.savemat(model_path[:index] + '/noise.mat', {'noise': noise.cpu().numpy()})  # Save noise to file
        noisy_img = img + noise
    else:
        noisy_img = img

    max_value = torch.max(img)  
    noisy_img_clipped = torch.clamp(noisy_img, 0, max_value).to(torch.uint8)
    print(noisy_img_clipped[1, :])
    cv2.imwrite(model_path[:index] + '/noisy.png', noisy_img_clipped.cpu().numpy().squeeze()) 
    
    # Adjust the complex field based on noise
    field = field_column.reshape(32, 32, 2).to(torch.float32).to('cuda')
    intensity = torch.sum(torch.abs(field) ** 2, dim=-1)
    new_intensity = intensity + noise.reshape(32, 32)
    
    magnitude = torch.sqrt(new_intensity)
    phase = torch.atan2(field[:,:,1], field[:,:,0])
    new_field = torch.stack((magnitude * torch.cos(phase), magnitude * torch.sin(phase)), dim=-1)
    
    return noisy_img.cpu().numpy(), new_field.cpu().numpy()  # Move results to CPU and convert to NumPy

# Configure the noise parameters and directories
sigma = 150  # Standard deviation for Gaussian noise
input_dir = 'data/train/output'
output_dir = 'data/train/noised_gaussian_150_L'
output_complex_dir = 'data/train/output_complex'
noise_dir = os.path.join(output_dir, 'noise_matrices')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(noise_dir, exist_ok=True)
os.makedirs(output_complex_dir, exist_ok=True)

# Process each image file
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
for idx, filename in enumerate(image_files):
    img_path = os.path.join(input_dir, filename)
    image = np.array(Image.open(img_path).convert('L'), dtype=np.float32)  # Load image and convert to grayscale
    model_path = os.path.join(noise_dir, filename)
    noised_image, new_field = add_gaussian_noise_and_complex_adjust(image, field1_train_t[:, idx, :], model_path, sigma)
    
    # Save the processed image and complex field
    output_img_path = os.path.join(output_dir, f'gaussian_noised_{filename}')
    Image.fromarray(noised_image.astype(np.uint8)).save(output_img_path)
    np.save(os.path.join(output_complex_dir, f'complex_{filename[:-4]}.npy'), new_field)

print("Gaussian noise added to all images and new complex matrices saved.")


In [ ]:
import torch
import numpy as np
import os
import cv2
from PIL import Image
import scipy.io as sio

def add_gaussian_noise_and_complex_adjust(img, field_column, base_path, sigma, noise_count=10):
    img_tensor = torch.from_numpy(img).to(torch.float32).to('cuda')
    noise_list = []
    
    # Create directories for each noise mask
    noise_dirs = [os.path.join(base_path, str(i + 1)) for i in range(noise_count)]
    for dir_path in noise_dirs:
        os.makedirs(dir_path, exist_ok=True)

    for i, noise_dir in enumerate(noise_dirs):
        noise = torch.normal(mean=0.0, std=sigma / 255., size=img_tensor.size(), device='cuda')
        noise_list.append(noise)
        noisy_img = img_tensor + noise
        noisy_img_clipped = torch.clamp(noisy_img, 0, 255).to(torch.uint8)
        cv2.imwrite(os.path.join(noise_dir, 'noisy.png'), noisy_img_clipped.cpu().numpy().squeeze())

    # Compute and handle the average noise
    average_noise = torch.mean(torch.stack(noise_list), dim=0)
    sio.savemat(base_path + '/average_noise.mat', {'noise': average_noise.cpu().numpy()})
    noisy_img_avg = img_tensor + average_noise
    noisy_img_avg_clipped = torch.clamp(noisy_img_avg, 0, 255).to(torch.uint8)
    cv2.imwrite(os.path.join(base_path, 'average_noised.png'), noisy_img_avg_clipped.cpu().numpy().squeeze())

    # Adjust the complex field based on average noise
    field = field_column.reshape(32, 32, 2).to(torch.float32).to('cuda')
    intensity = torch.sum(torch.abs(field) ** 2, dim=-1)
    new_intensity = intensity + average_noise.reshape(32, 32)
    magnitude = torch.sqrt(new_intensity)
    phase = torch.atan2(field[:, :, 1], field[:, :, 0])
    new_field = torch.stack((magnitude * torch.cos(phase), magnitude * torch.sin(phase)), dim=-1)

    return noisy_img_avg_clipped.cpu().numpy(), new_field.cpu().numpy()

sigma = 150  # Standard deviation for Gaussian noise
input_dir = 'data/train/output'
output_dir = 'data/train/noised_gaussian_150_L'
output_complex_dir = 'data/train/output_complex'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_complex_dir, exist_ok=True)

image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
for idx, filename in enumerate(image_files):
    img_path = os.path.join(input_dir, filename)
    image = np.array(Image.open(img_path).convert('L'), dtype=np.float32)
    base_path = os.path.join(output_dir, filename[:-4])  # Remove '.png' and use as base path
    noised_image_avg, new_field = add_gaussian_noise_and_complex_adjust(image, field1_train_t[:, idx, :], base_path, sigma)
    
    # Save the average noised image and complex field
    output_img_path = os.path.join(output_dir, f'average_gaussian_noised_{filename}')
    Image.fromarray(noised_image_avg.astype(np.uint8)).save(output_img_path)
    np.save(os.path.join(output_complex_dir, f'complex_avg_{filename[:-4]}.npy'), new_field)

print("Gaussian noise added to all images with multiple masks and averaged. New complex matrices saved.")


## Poisson

In [ ]:
import numpy as np
import os
from PIL import Image
import cv2
import scipy.io as sio

def add_poisson_noise(img, model_path, lam):
    index = model_path.rfind("/")
    lam = lam / 255
    if lam > 0:
        noise = np.random.poisson(lam, size=img.shape).astype(np.float32) - lam
        print(noise[1, :])
        sio.savemat(model_path[0:index] + '/noise.mat', {'noise': noise})
        noisy_img = img + noise
    else:
        noisy_img = img.astype(np.float32)
    cv2.imwrite(model_path[0:index] + '/noisy.png', np.squeeze(np.int32(np.clip(noisy_img, 0, 255))))
    return noisy_img

# Noise parameters
lam = 240  # Lambda parameter for the Poisson distribution

# Input and output directories
input_dir = 'data/val/output'
output_dir = 'data/train/noised_poisson_240'
noise_dir = os.path.join(output_dir, 'noise_matrices')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(noise_dir, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

# Process each image
for filename in image_files:
    # Read the image
    img_path = os.path.join(input_dir, filename)
    image = np.array(Image.open(img_path).convert('L'), dtype=np.float32)  # Convert image to grayscale for simplicity
    
    # Model path for saving noise matrix and noisy image
    model_path = os.path.join(noise_dir, filename)
    
    # Add Poisson noise to the image
    noised_image = add_poisson_noise(image, model_path, lam)
    
    # Save the processed image
    output_img_path = os.path.join(output_dir, f'poisson_noised_{filename}')
    Image.fromarray(np.uint8(noised_image)).save(output_img_path)

print("Poisson noise added to all images and saved.")


# Correlation

In [ ]:
import numpy as np

#file_path = 'data/train/output_corr_0/complex_train_speckle_0.npy'
file_path = 'MNIST/train/train_speckle.npy'

data = torch.load(file_path)

print("Shape of the data:", data.shape)
print("Data type of the data:", data.dtype)

print("First few elements of the data:", data)


In [ ]:
import numpy as np
import os

def compute_mu(field):
    N = field.shape[0]
    M = field.shape[1]
    mu = np.zeros((N, M), dtype=np.float32)

    for i in range(N):
        for j in range(M):
            if i == j:
                continue  # diag
            R_i, L_i = field[i, j, 0], field[i, j, 1]
            R_j, L_j = field[j, i, 0], field[j, i, 1]
            
            numerator = R_i * R_j + L_i * L_j
            denominator = np.sqrt((R_i**2 + L_i**2) * (R_j**2 + L_j**2))
            if denominator != 0:
                mu[i, j] = numerator / denominator
    
    return mu

input_dir = 'data/train/output_complex'
output_dir = 'data/train/output_corr_0'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(input_dir, filename)
        complex_field = np.load(file_path)
        
        mu_matrix = compute_mu(complex_field.reshape(128, 128, 2)) 
        
        output_file_path = os.path.join(output_dir, filename)
        np.save(output_file_path, mu_matrix)

print("Correlation matrices (mu values) have been calculated and saved.")


## Without Diag

In [ ]:
import numpy as np
import os
import cupy as cp

def compute_mu(field):
    N = field.shape[0] * field.shape[1] 
    flat_field = field.reshape(N, 2)  # flatten matrix
    flat_field = cp.asarray(flat_field)
    
    mu = cp.zeros((N, N - 1), dtype=cp.float32)
    
    for i in range(N):
        print(i)
        k = 0  
        for j in range(N):
            if i != j:
                R_i, L_i = flat_field[i]
                R_j, L_j = flat_field[j]
                
                numerator = R_i * R_j + L_i * L_j
                denominator = cp.sqrt((R_i**2 + L_i**2) * (R_j**2 + L_j**2))
                if denominator != 0:
                    mu[i, k] = numerator / denominator
                k += 1
    
    mu = cp.asnumpy(mu)
    return mu

input_dir = 'data/train/output_complex_0'
output_dir = 'data/train/output_corr_0'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(input_dir, filename)
        complex_field = np.load(file_path)
        complex_field = cp.asarray(complex_field)
        
        mu_matrix = compute_mu(complex_field.reshape(32, 32, 2))  
        
        output_file_path = os.path.join(output_dir, filename)
        np.save(output_file_path, mu_matrix)

print("Correlation matrices (mu values) have been calculated and saved.")


## With Diag

In [ ]:
import numpy as np
import os

import cupy as cp

def compute_mu(field):
    N = field.shape[0] * field.shape[1] 
    flat_field = field.reshape(N, 2)

    flat_field = cp.asarray(flat_field)
    mu = cp.zeros((N, N), dtype=cp.float32)
    
    cp.fill_diagonal(mu, 1)

    for i in range(N):
        print(i)
        for j in range(i+1, N):  # calculate only upper triangular matrix
            R_i, L_i = flat_field[i]
            R_j, L_j = flat_field[j]
            
            numerator = R_i * R_j + L_i * L_j
            denominator = cp.sqrt((R_i**2 + L_i**2) * (R_j**2 + L_j**2))
            if denominator != 0:
                mu[i, j] = mu[j, i] = numerator / denominator
    
    mu = cp.asnumpy(mu)
    return mu

input_dir = 'data/train/output_complex_0'
output_dir = 'data/train/output_corr_00'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(input_dir, filename)
        complex_field = np.load(file_path)
        complex_field = cp.asarray(complex_field)
        
        mu_matrix = compute_mu(complex_field.reshape(32, 32, 2)) 
        
        output_file_path = os.path.join(output_dir, filename)
        np.save(output_file_path, mu_matrix)

print("Correlation matrices (mu values) have been calculated and saved.")


# Probability

In [ ]:
import numpy as np
import os
from PIL import Image
from scipy.special import i0  # Bessel function of the first kind

def process_image(file_path):
    img = Image.open(file_path).convert('L')  # grayscale
    img_array = np.array(img)
    print("Original image shape:", img_array.shape)  # size of the original images
    
    # remove even columns
    processed_array = img_array[:, 1::2]
    processed_array = np.clip(processed_array, 1e-8, 255)
    # flatten
    flattened_vector = processed_array.flatten()
    print("Processed image shape:", processed_array.shape)  # image size after processing
    print("Flattened vector length:", len(flattened_vector))  # length of flattened vector
    return flattened_vector

def process_matrix(file_path):
    matrix = np.load(file_path)
    
    # remove odd columns and even rows
    processed_matrix = matrix[1::2, ::2]
    print("Processed matrix shape:", processed_matrix.shape)

    lower_bound = -1 + 1e-8
    upper_bound = 1 - 1e-8
    processed_matrix = np.clip(processed_matrix, lower_bound, upper_bound)
    
    return processed_matrix

def calculate_probability(vector, matrix):
    probabilities = []
    
    for i, I_i in enumerate(vector):
        M_i = matrix[i, :]  # ith row
        numerator = np.exp(- (1 + M_i**2) / (1 - M_i**2)) * i0(2 * M_i / (1 - M_i**2))
        denominator = I_i * (1 - M_i**2)
            
        p_i_M = numerator / denominator
        p_i_M[np.isinf(p_i_M)] = np.nan
        p_i_M = np.clip(p_i_M, -1, 1)
        p_i_M_avg = np.nanmean(p_i_M)  # avg prob of the ith row, ignore NaN and inf
        probabilities.append(p_i_M_avg)
    
    return np.nanmean(probabilities)  # avg prob, ignore NaN and inf


image_dir = 'data/train/output_temp'
matrix_dir = 'data/train/output_corr'
results = []
   
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
matrix_files = sorted([f for f in os.listdir(matrix_dir) if f.endswith('.npy')])
    
matches = {img_file: None for img_file in image_files}
for img_file in image_files:
    index = img_file.split('_')[-1].split('.')[0]  # indices after _k
    for mat_file in matrix_files:
        if mat_file.endswith(f'_{index}.npy'):
            matches[img_file] = mat_file
            break
    
for img_file, mat_file in matches.items():
    vector = process_image(os.path.join(image_dir, img_file))
    matrix = process_matrix(os.path.join(matrix_dir, mat_file))
        
    probability = calculate_probability(vector, matrix)
    results.append(probability)

for result in results:
    print(result)


In [ ]:
import numpy as np
import os
from PIL import Image
from scipy.special import i0  # Bessel function of the first kind

def process_image(file_path):
    img = Image.open(file_path).convert('L')  # grayscale
    img_array = np.array(img)
    print("Original image shape:", img_array.shape) 
    
    # remove even columns
    processed_array = img_array[:, 1::2]
    processed_array = np.clip(processed_array, 1e-8, 255)
    flattened_vector = processed_array.flatten()
    print("Processed image shape:", processed_array.shape) 
    print("Flattened vector length:", len(flattened_vector)) 
    return flattened_vector

def process_matrix(file_path):
    matrix = np.load(file_path)
    
    # remove odd columns and even rows
    processed_matrix = matrix[1::2, ::2]
    print("Processed matrix shape:", processed_matrix.shape)

    lower_bound = -1 + 1e-8
    upper_bound = 1 - 1e-8
    processed_matrix = np.clip(processed_matrix, lower_bound, upper_bound)
    
    return processed_matrix

def calculate_probability(vector, matrix):
    probabilities = []
    
    for i, I_i in enumerate(vector):
        M_i = matrix[i, :]  # ith row
        numerator = np.exp(- (1 + M_i**2) / (1 - M_i**2)) * i0(2 * M_i / (1 - M_i**2))
        denominator = I_i * (1 - M_i**2)
            
        p_i_M = numerator / denominator
        p_i_M[np.isinf(p_i_M)] = np.nan
        p_i_M = np.clip(p_i_M, -1, 1)
        p_i_M_avg = np.nanmean(p_i_M)  # avg prob of the ith row, ignore NaN and inf
        probabilities.append(p_i_M_avg)
    
    return np.nanmean(probabilities)  # avg prob


image_dir = 'data/train/avgnoise_pred'
matrix_dir = 'data/train/output_corr_0'
results = []
   
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
matrix_files = sorted([f for f in os.listdir(matrix_dir) if f.endswith('.npy')])
    
matches = {img_file: None for img_file in image_files}
for img_file in image_files:
    index = img_file.split('_')[-1].split('.')[0]  
    for mat_file in matrix_files:
        if mat_file.endswith(f'_{index}.npy'):
            matches[img_file] = mat_file
            break
    
for img_file, mat_file in matches.items():
    vector = process_image(os.path.join(image_dir, img_file))
    matrix = process_matrix(os.path.join(matrix_dir, mat_file))
        
    probability = calculate_probability(vector, matrix)
    results.append((img_file, probability)) 

for img_name, result in results:
    print(f"{img_name}: {result}")


# Line Chart

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data1 = np.loadtxt('prob.txt', delimiter=',')
data2 = np.loadtxt('prob_2.txt', delimiter=',')

x = data1[:, 0] 
y1 = data1[:, 1]  
y2 = data2[:, 1]  

plt.figure(figsize=(10, 6))
plt.plot(x, y1, marker='o', linestyle='-', color='blue', label='prob.txt')
plt.plot(x, y2, marker='x', linestyle='--', color='red', label='prob_2.txt')
plt.title('Probability Comparisons')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import numpy as np
import os
from PIL import Image
from scipy.special import i0  # Bessel function of the first kind

def process_image(file_path):
    img = Image.open(file_path).convert('L')  # grayscale
    img_array = np.array(img)
    print("Original image shape:", img_array.shape)  
    
    # remove even columns
    processed_array = img_array[:, 1::2]
    processed_array = np.clip(processed_array, 1e-8, 255)

    flattened_vector = processed_array.flatten()
    print("Processed image shape:", processed_array.shape)  
    print("Flattened vector length:", len(flattened_vector)) 
    return flattened_vector

def process_matrix(file_path):
    matrix = np.load(file_path)
    
    # remove odd columns and even rows
    processed_matrix = matrix[1::2, ::2]
    print("Processed matrix shape:", processed_matrix.shape)

    lower_bound = -1 + 1e-8
    upper_bound = 1 - 1e-8
    processed_matrix = np.clip(processed_matrix, lower_bound, upper_bound)
    
    return processed_matrix

def calculate_probability(vector, matrix):
    probabilities = []
    
    for i, I_i in enumerate(vector):
        M_i = matrix[i, :]  
        numerator = np.exp(- (1 + M_i**2) / (1 - M_i**2)) * i0(2 * M_i / (1 - M_i**2))
        denominator = I_i * (1 - M_i**2)
            
        p_i_M = numerator / denominator
        p_i_M[np.isinf(p_i_M)] = np.nan
        p_i_M = np.clip(p_i_M, -1, 1)
        p_i_M_avg = np.nanmean(p_i_M)  
        probabilities.append(p_i_M_avg)
    
    return np.nanmean(probabilities)  


image_dir = 'data/train/noise_pred'
matrix_dir = 'data/train/output_corr_0'
results = []
   
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
matrix_files = sorted([f for f in os.listdir(matrix_dir) if f.endswith('.npy')])
    
matches = {img_file: None for img_file in image_files}
for img_file in image_files:
    index = img_file.split('_')[-1].split('.')[0]
    for mat_file in matrix_files:
        if mat_file.endswith(f'_{index}.npy'):
            matches[img_file] = mat_file
            break
    
for img_file, mat_file in matches.items():
    vector = process_image(os.path.join(image_dir, img_file))
    matrix = process_matrix(os.path.join(matrix_dir, mat_file))
        
    probability = calculate_probability(vector, matrix)
    results.append(probability)

for result in results:
    print(result)


In [ ]:
import numpy as np
from PIL import Image
import os

def bessel_I0(z):
    if z <= 0.5:
        return 1
    else:
        return np.exp(z) / np.sqrt(2 * np.pi * z)

def compute_pdf(image_path, mu_matrix_path):
    image = np.array(Image.open(image_path).convert('L'), dtype=np.float32)
    mu_matrix = np.load(mu_matrix_path)
    
    N = 128  # calculate only first 128 pixels
    probabilities = np.zeros(N)
    
    for i in range(N):
        prob_sum = 0
        count = 0
        for j in range(16384): # 128*128 
            if i == j:
                continue
            
            I_i = image[i // image.shape[1], i % image.shape[1]]
            I_j = image[j // image.shape[1], j % image.shape[1]]
            mu_ij = mu_matrix[i // image.shape[1], j % image.shape[1]]
            if mu_ij <= -1:
                mu_ij = -0.99
            if mu_ij >= 1:
                mu_ij = 0.99
            
            if I_j == 0 or 1 - mu_ij**2 == 0:
                continue  
            
            exp_argument = (1 + mu_ij**2) / (1 - mu_ij**2)
            z = 2 * mu_ij / (1 - mu_ij**2)
            I0_val = bessel_I0(z)
            p = 1 / (I_j * (1 - mu_ij**2)) * np.exp(-exp_argument) * I0_val
            
            if np.isfinite(p): 
                prob_sum += p
                count += 1

        probabilities[i] = prob_sum / count if count > 0 else 0

    # avg prob excluding NaN and inf
    valid_probs = probabilities[np.isfinite(probabilities)]
    valid_prob_avg = np.mean(valid_probs) if valid_probs.size > 0 else 0
    return valid_prob_avg, probabilities

input_dir = 'data/train/noised_gaussian_150_L'
mu_dir = 'data/train/output_corr'
output_probabilities = []
probs = []

# avg prob for each image
for x in range(2,10):
    image_path = os.path.join(input_dir, f'gaussian_noised_train_speckle_{x}.png')
    mu_path = os.path.join(mu_dir, f'complex_train_speckle_{x}.npy')
    avg_valid_prob, prob = compute_pdf(image_path, mu_path)
    output_probabilities.append(avg_valid_prob)
    probs.append(prob)

for idx, avg_prob in enumerate(output_probabilities):
    print(f"Average valid conditional probabilities for image {idx}: {avg_prob}")
    
for idx, pdf in enumerate(probs):
    print(f"Average conditional probabilities for image {idx}: {pdf}")


In [ ]:
def bessel_I0(x):
    """Approximate the modified Bessel function of the first kind, I_0(x), using a series expansion."""
    terms = 10  # Number of terms in the series expansion
    result = tf.ones_like(x)
    factorial = tf.ones_like(x)
    x_squared = tf.square(x / 2)
    power_of_x_squared = tf.ones_like(x)
    
    for k in range(1, terms + 1):
        factorial *= k
        power_of_x_squared *= x_squared
        result += power_of_x_squared / (factorial * factorial)
    
    return result

def compute_conditional_probabilities(response, mask_tensor, mu_matrix):
    mu_matrix = tf.convert_to_tensor(mu_matrix, dtype=tf.float32)
    
    # Expand dimensions to match the response tensor shape for broadcasting
    mu_matrix = tf.expand_dims(mu_matrix, 0)  # Assuming batch dimension is required
    
    # Get dimensions for indexing and calculations
    N = tf.shape(response)[-1]  # assuming the last dimension is the spatial dimension

    # Flatten response and mask to simplify the calculation
    response_flat = tf.reshape(response, [-1, N])
    mask_flat = tf.reshape(mask_tensor, [-1, N])

    # Compute probabilities for all pairs (i, j)
    # Prepare mu_ij values, ensuring they are within the limits [-1 + eps, 1 - eps] to avoid division by zero in 1 - mu_ij^2
    eps = 1e-6
    mu_ij = tf.clip_by_value(mu_matrix, -1 + eps, 1 - eps)

    exp_argument = (1 + mu_ij**2) / (1 - mu_ij**2)
    z = 2 * mu_ij / (1 - mu_ij**2)
    I0_val = bessel_I0(z)

    # Calculate the actual probabilities
    I_j = tf.expand_dims(response_flat, 2)  # Create a new axis for j
    probabilities = 1 / (I_j * (1 - mu_ij**2)) * tf.exp(-exp_argument) * I0_val
    
    # Use mask to adjust where j is not used in the computation
    mask_j = tf.expand_dims(mask_flat, 2)  # Apply mask
    probabilities *= mask_j  # Apply mask to exclude certain values

    # Check for finite probabilities and ignore NaNs or Infs
    probabilities = tf.where(tf.math.is_finite(probabilities), probabilities, tf.zeros_like(probabilities))

    # Sum probabilities over all j for each i, ignoring i itself (diagonal elements)
    diagonal_mask = 1 - tf.eye(N)  # Create a mask to zero-out diagonal (self-references)
    diagonal_mask = tf.expand_dims(diagonal_mask, 0)  # Adjust shape for batch
    probabilities *= diagonal_mask  # Apply diagonal mask

    # Average the probabilities over all j for each i
    probabilities_sum = tf.reduce_sum(probabilities, axis=2)
    count_valid = tf.reduce_sum(diagonal_mask * mask_j, axis=2)  # count valid terms only
    avg_probabilities = probabilities_sum / count_valid

    # Finally, average over all i to get a single probability value per image in the batch
    final_prob = tf.reduce_mean(avg_probabilities, axis=1)

    return final_prob

def build_denoising_unet(noisy, mu_matrix, p=0.7, is_realnoisy=False, a=0.1):
    _, h, w, c = np.shape(noisy)
    noisy_tensor = tf.identity(noisy)
    is_flip_lr = tf.placeholder(tf.int16)
    is_flip_ud = tf.placeholder(tf.int16)
    noisy_tensor = data_arg(noisy_tensor, is_flip_lr, is_flip_ud)
    response = tf.transpose(noisy_tensor, [0, 3, 1, 2])
    mask_tensor = tf.ones_like(response) * 0.7  # Assuming dropout applies here for simplicity

    response = tf.multiply(mask_tensor, response)
    if is_realnoisy:
        response = tf.squeeze(tf.random_poisson(25 * response, [1]) / 25, 0)

    response = partial_conv_unet(response, mask_tensor, channel=c, width=w, height=h, p=p)
    response = tf.transpose(response, [0, 2, 3, 1])
    mask_tensor = tf.transpose(mask_tensor, [0, 2, 3, 1])

    response = data_arg(response, is_flip_lr, is_flip_ud)

    # Compute the conditional probabilities as an additional loss term
    conditional_prob = tf.reduce_mean(compute_conditional_probabilities(response, mask_tensor, mu_matrix))
    print(conditional_prob)
    original_loss = mask_loss(response, noisy_tensor, 1. - mask_tensor)
    data_loss = original_loss - a * conditional_prob
    data_loss = tf.maximum(data_loss, 0)  # Ensuring loss doesn't go below 0
    print("Data loss shape:", data_loss.get_shape())

    # Maintain a running average of the output images
    slice_avg = tf.get_variable('slice_avg', shape=[_, h, w, c], initializer=tf.initializers.zeros())
    avg_op = slice_avg.assign(slice_avg * 0.99 + response * 0.01)

    training_error = data_loss
    tf.summary.scalar('data loss', data_loss)

input_dir = 'data/train/output'
mu_dir = 'data/train/output_corr'
output_probabilities = []
probs = []


for x in range(10):  # the 10th image
    image_path = os.path.join(input_dir, f'train_speckle_{x}.png')
    mu_path = os.path.join(mu_dir, f'complex_train_speckle_{x}.npy')
    mu_matrix = np.load(mu_path)
    noisy = util.load_np_image(file_path)
    conditional_prob =  build_denoising_unet(noisy, mu_matrix, p=0.7, is_realnoisy=False, a=0.1)
    output_probabilities.append(conditional_prob)

for idx, avg_prob in enumerate(output_probabilities):
    print(f"Average valid conditional probabilities for image {idx}: {avg_prob}")

# Down-sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import maximum_filter

def max_pool_and_expand(image):
    # max pooling in 2*2
    pooled = maximum_filter(image, size=2, footprint=np.ones((2, 2)), mode='reflect')
    # pick only the top-left pixel
    pooled = pooled[::2, ::2]
    # expand to 2*2 blocks by kronecker
    expanded_image = np.kron(pooled, np.ones((2, 2)))
    return expanded_image

pooled_train_images = [max_pool_and_expand(img) for img in processed_train_images]
pooled_val_images = [max_pool_and_expand(img) for img in processed_val_images]

plt.imshow(pooled_train_images[0], cmap='gray')
plt.title('Pooled Training Image')
plt.axis('off')
plt.show()

plt.imshow(pooled_val_images[0], cmap='gray')
plt.title('Pooled Validation Image')
plt.axis('off')
plt.show()


In [ ]:
Nslm = 16384  # nb of SLM pixels
Nfluo = 16384   # nb of beads
Ncam = 16384  # nb of camera pixels

pooled_train_images_flat = [img.reshape(-1) for img in pooled_train_images] 
pooled_val_images_flat = [img.reshape(-1) for img in pooled_val_images]

SLM_input_train_pooled = np.stack(pooled_train_images_flat, axis=1)
SLM_input_val_pooled = np.stack(pooled_val_images_flat, axis=1)

SLM_input_train_pha_pooled = torch.from_numpy(SLM_input_train_pooled).float().to(device)
SLM_input_train_t_pooled = phase_to_complex(SLM_input_train_pha_pooled)

SLM_input_val_pha_pooled = torch.from_numpy(SLM_input_val_pooled).float().to(device)
SLM_input_val_t_pooled = phase_to_complex(SLM_input_val_pha_pooled)

field1_train_t_pooled = complex_matmul(T1_t, SLM_input_train_t_pooled)
int1_train_t_pooled = torch.sum(torch.abs(field1_train_t_pooled)**2,-1)

field1_val_t_pooled = complex_matmul(T1_t, SLM_input_val_t_pooled)
int1_val_t_pooled = torch.sum(torch.abs(field1_val_t_pooled)**2,-1)
# int1_val_t = int1_val_t.T
# CAM_output_t = T2_t@int1_t

In [ ]:
import os

np.save('down-sampled/train/train_image.npy', SLM_input_train_pooled)

torch.save(int1_train_t_pooled.cpu(), 'down-sampled/train/train_speckle.npy')

In [ ]:
np.save('down-sampled/val/val_image.npy', SLM_input_val_pooled)

torch.save(int1_val_t_pooled.cpu(), 'down-sampled/val/val_speckle.npy')

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

output_dir = 'down-sampled/train/input'
os.makedirs(output_dir, exist_ok=True)

for idx, img in enumerate(pooled_train_images):
    img = (img - img.min()) / (img.max() - img.min())  
    img = (img * 255).astype(np.uint8)  

    image = Image.fromarray(img)

    file_path = os.path.join(output_dir, f'train_image_{idx}.png')
    # saved as .tif
    image.save(file_path)

print(f"Saved {len(pooled_train_images)} images to {output_dir}")

file_name = 'train_image_0.png'
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray')  # grayscale
plt.title(f'Displaying {file_name}')
plt.axis('off') 
plt.show()

In [ ]:
output_dir = 'down-sampled/val/input'
os.makedirs(output_dir, exist_ok=True)

for idx, img in enumerate(pooled_val_images):
    img = (img - img.min()) / (img.max() - img.min()) 
    img = (img * 255).astype(np.uint8) 

    image = Image.fromarray(img)
    file_path = os.path.join(output_dir, f'val_image_{idx}.png')
    image.save(file_path)

print(f"Saved {len(pooled_val_images)} images to {output_dir}")

file_name = 'val_image_0.png'
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray') 
plt.title(f'Displaying {file_name}')
plt.axis('off') 
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt
import os

speckle_data = torch.load('down-sampled/train/train_speckle.npy')

output_dir = 'down-sampled/train/output'
os.makedirs(output_dir, exist_ok=True)

for i in range(speckle_data.shape[1]): 
    image = speckle_data[:, i].reshape(128, 128).numpy() 
    
    image_normalized = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    image_scaled = (image_normalized * 255).astype(np.uint8)
    
    output_filename = os.path.join(output_dir, f'train_speckle_{i}.png')
    
    plt.imsave(output_filename, image_scaled, cmap='gray')

file_name = 'train_speckle_0.png' 
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray') 
plt.title(f'Displaying {file_name}')
plt.axis('off') 
plt.show()


In [ ]:
speckle_data = torch.load('down-sampled/val/val_speckle.npy')

output_dir = 'down-sampled/val/output'
os.makedirs(output_dir, exist_ok=True)

for i in range(speckle_data.shape[1]): 
    image = speckle_data[:, i].reshape(128, 128).numpy()
    
    image_normalized = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    image_scaled = (image_normalized * 255).astype(np.uint8)
    
    output_filename = os.path.join(output_dir, f'val_speckle_{i}.png')
    
    plt.imsave(output_filename, image_scaled, cmap='gray')

file_name = 'val_speckle_0.png'
file_path = os.path.join(output_dir, file_name)

image = Image.open(file_path)

plt.imshow(image, cmap='gray') 
plt.title(f'Displaying {file_name}')
plt.axis('off')  
plt.show()